In [1]:
%cd

/root


In [2]:
!git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.81 MiB | 28.74 MiB/s, done.
Resolving deltas: 100% (570/570), done.


In [0]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
# # Import Mask RCNN
# sys.path.append('/content')  # To find local version of the library
%cd ~/Mask_RCNN
ROOT_DIR = os.path.abspath("../")

/root/Mask_RCNN


In [6]:
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

Using TensorFlow backend.


In [0]:
# Import COCO config
sys.path.append(os.path.join("/root/Mask_RCNN/samples/coco/"))  # To find local version
import coco

In [0]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [9]:
# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('', "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

... done downloading pretrained model!


In [0]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()


In [11]:
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [0]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)

# Load weights trained on MS-COCO
model.load_weights('mask_rcnn_coco.h5', by_name=True)

In [0]:
# COCO Class names
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']
# COCO Class names
# class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
#                'bus', 'train', 'truck', 'boat', 'traffic light',
#                'fire hydrant', 'stop sign', 'parking meter','bird',
#                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear']

In [0]:
import cv2
import time

In [0]:
def display_results(image, boxes, masks, class_ids, class_names, scores=None,
                        show_mask=True, show_bbox=True, display_img=True,
                        save_img=True, save_dir=None, img_name=None):
        """
        boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
        masks: [height, width, num_instances]
        class_ids: [num_instances]
        class_names: list of class names of the dataset (Without Background)
        scores: (optional) confidence scores for each box
        show_mask, show_bbox: To show masks and bounding boxes or not
        display_img: To display the image in popup
        save_img: To save the predict image
        save_dir: If save_img is True, the directory where you want to save the predict image
        img_name: If save_img is True, the name of the predict image

        """
        n_instances = boxes.shape[0]
        colors = color_map()
        for k in range(n_instances):
            color = colors[class_ids[k]].astype(np.int)
            if show_bbox:
                box = boxes[k]
                cls = class_names[class_ids[k]]  # Skip the Background
                score = scores[k]
                cv2.rectangle(image, (box[1], box[0]), (box[3], box[2]), 1)
                font = cv2.FONT_HERSHEY_SIMPLEX
                cv2.putText(image, '{}: {:.3f}'.format(cls, score), (box[1], box[0]),
                            font, 0.4, (0, 255, 255), 1, cv2.LINE_AA)

            if show_mask:
                mask = masks[:, :, k]
                color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.int)
                color_mask[mask] = color
                image = cv2.addWeighted(color_mask, 0.5, image.astype(np.int), 1, 0)

        if display_img:
            plt.imshow(image)
            plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
            plt.show()
        if save_img:
            cv2.imwrite(os.path.join(save_dir, img_name), image)

        return None

In [0]:
def color_map(N=256, normalized=False):
        def bitget(byteval, idx):
            return ((byteval & (1 << idx)) != 0)

        dtype = 'float32' if normalized else 'uint8'
        cmap = np.zeros((N, 3), dtype=dtype)
        for i in range(N):
            r = g = b = 0
            c = i
            for j in range(8):
                r = r | (bitget(c, 0) << 7 - j)
                g = g | (bitget(c, 1) << 7 - j)
                b = b | (bitget(c, 2) << 7 - j)
                c = c >> 3

            cmap[i] = np.array([r, g, b])

        cmap = cmap / 255 if normalized else cmap
        return cmap

In [0]:
# # Load a random image from the images folder
image = skimage.io.imread('/root/Mask_RCNN/test1.jpg')

# original image
plt.figure(figsize=(12,10))
skimage.io.imshow(image)

In [0]:
ROOT_DIR = os.getcwd()

VIDEO_IMAGE_DIR = '/root/Mask_RCNN/processedIMG'
VIDEO_SAVE_DIR = os.path.join(ROOT_DIR, 'NewVideo')
VIDEO_FILE_DIR = '/root/Mask_RCNN'
os.mkdir(VIDEO_SAVE_DIR)
os.mkdir(VIDEO_IMAGE_DIR)

In [0]:
# Run detection
results = model.detect([image], verbose=1)
# # Visualize results
r = results[0]
display_results(image, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'],show_mask=False,display_img=False, save_dir= VIDEO_FILE_DIR,img_name = 'test1_out.jpg')

# plt.savefig('/root/Mask_RCNN/test3_Out.jpg',bbox_inches='tight', pad_inches=-0.5,orientation= 'landscape')

In [0]:
%cd



---



---



---



---



In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [23]:
pip install pytube

In [25]:
%cd /root/Mask_RCNN

/root/Mask_RCNN


In [59]:
#importing the module 
import pytube
  
#where to save 
SAVE_PATH = "/content" #to_do 
  
#link of the video to be downloaded https://youtu.be/SuWEtGYhdQs
link = "https://www.youtube.com/watch?v=pk96gqasGBQ"
# link="https://www.youtube.com/watch?v=Y1jTEyb3wiI"
  
try: 
    #object creation using YouTube which was imported in the beginning 
    yt =pytube.YouTube(link) 
except: 
    print("Connection Error") #to handle exception 
  
# #filters out all the files with "mp4" extension 
# mp4files = yt.filter('mp4') 
stream = yt.streams.first()
stream.download()  


'/root/Mask_RCNN/Test video for Object Detection  TRIDE.mp4'

In [0]:
frame_count = 0
batch_size = 1
frames = []
capture = cv2.VideoCapture(os.path.join(VIDEO_FILE_DIR, 'test_video_1.mp4'))    
while True:
    ret, frame = capture.read()
    # Bail out when the video file ends
    if not ret:
        break        
    # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    if len(frames) == batch_size:
        results = model.detect(frames, verbose=0)
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            # frame = visualize.display_instances(
            #     frame, r['rois'], r['masks'], r['class_ids'], class_names, r['scores']
            # );
            print(frame_count)
            name = '{0}.jpg'.format(frame_count + i - batch_size)
            name = os.path.join(VIDEO_IMAGE_DIR, name)
            display_results(frame, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'],show_mask=False,display_img=False,
                            save_img=True, save_dir= VIDEO_IMAGE_DIR,img_name = name)
            cv2.imwrite(name, frame)
        # Clear the frames array to start the next batch
    frames = []

In [62]:

video = cv2.VideoCapture(os.path.join(VIDEO_FILE_DIR, 'test_video_1.mp4')) 

# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
fps = video.get(cv2.CAP_PROP_FPS)
print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))


Frames per second using video.get(cv2.CAP_PROP_FPS) : 25.0


In [0]:
import glob

In [0]:
# Get all image file paths to a list.
images = list(glob.iglob(os.path.join(VIDEO_IMAGE_DIR, '*.*')))
# Sort the images by name index.
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))
images;

In [65]:
img = cv2.imread(images[1])
np.shape(img)

(720, 1280, 3)

In [0]:
def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

import glob
import os


In [68]:
%cd /root/Mask_RCNN

/root/Mask_RCNN


In [69]:
# Directory of images to run detection on
# ROOT_DIR = os.getcwd()
# VIDEO_IMAGE_DIR = '/root/Mask_RCNN/imagesFromVideo'
# VIDEO_SAVE_DIR = os.path.join(ROOT_DIR, "video/save")
# images = list(glob.iglob(os.path.join(VIDEO_IMAGE_DIR, '*.jpg')))
# # Sort the images by integer index
# images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

# outvid = os.path.join(VIDEO_SAVE_DIR,'out.mp4')
make_video(outvid, images=images, fps=30, size=None,
               is_color=True, format="FMP4")

<VideoWriter 0x7efb5243d130>

In [0]:
from google.colab import files
files.download(outvid) 